# Many to Many Relationship In DynamoDB - Method 1 Adjacency List

Reference:

- [Stackovervlow - How to model one-to-one, one-to-many and many-to-many relationships in DynamoDB](https://stackoverflow.com/questions/55152296/how-to-model-one-to-one-one-to-many-and-many-to-many-relationships-in-dynamodb)
- [AWS - Best practices for managing many-to-many relationships](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/bp-adjacency-graphs.html)
- [AWS - Best practices for modeling relational data in DynamoDB](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/bp-relational-modeling.html)

# Use Case

- One Student enrolls many courses.
- One Course has many students.

# Import Python Libraries

In [1]:
import typing as T
import enum
from datetime import datetime

import dataclasses
import pynamodb_mate as pm
import pynamodb.exceptions as exc
from moto import mock_dynamodb

from rich import print as rprint

# Configure AWS Connection

In [2]:
# create a DynamoDB connection, ensure that your default AWS credential is right
# if you are using mock, then this line always works
connect = pm.Connection()

In [3]:
# use moto to mock DynamoDB, it is an in-memory implementation of DynamoDB
# you can also use the real DynamoDB table by just comment out the below two line
mock = mock_dynamodb()
mock.start()

# Type Hint

In [4]:
REQUIRED_STR = T.Union[str, pm.UnicodeAttribute]
OPTIONAL_STR = T.Optional[REQUIRED_STR]
REQUIRED_INT = T.Union[int, pm.NumberAttribute]
OPTIONAL_INT = T.Optional[REQUIRED_INT]
REQUIRED_DATETIME = T.Union[datetime, pm.UTCDateTimeAttribute]
OPTIONAL_DATETIME = T.Optional[REQUIRED_DATETIME]

# Declare Student

In [5]:
class LookupIndex(pm.GlobalSecondaryIndex):
    class Meta:
        index = "lookup-index"
        projection = pm.AllProjection

    sk: REQUIRED_STR = pm.UnicodeAttribute(hash_key=True)


class Entity(pm.Model):
    class Meta:
        table_name = "entity"
        region = "us-east-1"
        billing_mode = pm.PAY_PER_REQUEST_BILLING_MODE

    pk: REQUIRED_STR = pm.UnicodeAttribute(hash_key=True)
    sk: REQUIRED_STR = pm.UnicodeAttribute(range_key=True)

    type: REQUIRED_STR = pm.UnicodeAttribute()
    name: OPTIONAL_STR = pm.UnicodeAttribute(null=True)
    enroll_date: OPTIONAL_STR = pm.UnicodeAttribute(null=True)
    
    lookup_index = LookupIndex()
    

class Student(Entity):
    lookup_index = LookupIndex()
    
    @property
    def student_name(self) -> str:
        return self.name


class Course(Entity):
    lookup_index = LookupIndex()
    
    @property
    def course_name(self) -> str:
        return self.name
        

Entity.create_table(wait=True)

# Business Operation

In [6]:
class OP:
    @classmethod
    def new_student(
        cls,
        student_id: str, 
        student_name: str,
    ) -> T.Optional["Student"]:
        student = Student(
            pk=student_id,
            sk=student_id,
            type="student",
            name=student_name,
        )
        try:
            res = student.save(
                condition= (~Student.pk.exists()),
            )
            return student
        except exc.PutError as e:
            return None

    @classmethod
    def new_course(
        cls,
        course_id: str, 
        course_name: str,
    ) -> T.Optional["Student"]:
        course = Course(
            pk=course_id,
            sk=course_id,
            type="course",
            name=course_name,
        )
        try:
            res = course.save(
                condition= (~Course.pk.exists()),
            )
            return course
        except exc.PutError as e:
            return None
            
    def all_student(self) -> list[Student]:
        return list(Student.scan(filter_condition=Student.type == "student"))

    def all_course(self) -> list[Student]:
        return list(Course.scan(filter_condition=Course.type == "course"))

    def all_entity(self) -> list[Entity]:
        return list(Entity.scan())
    
    def enroll(self, student_id: str, course_id: str):
        student = Student(
            pk=student_id, 
            sk=course_id, 
            type="enrolled_course",
            enroll_date=str(datetime.now().date()),
        )
        try:
            res = student.save(
                condition= ~(Student.pk.exists() & Student.sk.exists()),
            )
            return student
        except exc.PutError as e:
            return None

    def find_all_student_in_given_course(self, course_id: str) -> list[Student]:
        return list(
            LookupIndex.query(
                hash_key=course_id,
                filter_condition=Entity.type == "enrolled_course",
            ),
        )

    def find_all_enrolled_course(self, student_id: str) -> list[Student]:
        return list(
            Student.query(
                hash_key=student_id,
                filter_condition=Entity.type == "enrolled_course",
            ),
        )

op = OP()

# Create Dummy Data

In [7]:
s1 = op.new_student(student_id="s-1", student_name="Alice")
s2 = op.new_student(student_id="s-2", student_name="Bob")
s3 = op.new_student(student_id="s-3", student_name="Cathy")

c1 = op.new_course(course_id="c-1", course_name="Math")
c2 = op.new_course(course_id="c-2", course_name="Science")

# Show all Data

In [8]:
rprint("------ All Student ------")
for student in op.all_student():
    rprint(student.to_dict())
rprint("------ Course ------")
for course in op.all_course():
    rprint(course.to_dict())

------ All Student ------

{'name': 'Alice', 'pk': 's-1', 'sk': 's-1', 'type': 'student'}

{'name': 'Bob', 'pk': 's-2', 'sk': 's-2', 'type': 'student'}

{'name': 'Cathy', 'pk': 's-3', 'sk': 's-3', 'type': 'student'}

------ Course ------

{'name': 'Math', 'pk': 'c-1', 'sk': 'c-1', 'type': 'course'}

{'name': 'Science', 'pk': 'c-2', 'sk': 'c-2', 'type': 'course'}

# Student Enroll Course

In [9]:
enroll = op.enroll(student_id="s-1", course_id="c-1")
enroll = op.enroll(student_id="s-2", course_id="c-1")
enroll = op.enroll(student_id="s-2", course_id="c-2")
enroll = op.enroll(student_id="s-3", course_id="c-2")

rprint("------ All Data ------")
for entity in op.all_entity():
    rprint(f"{entity.type = }, pk = {entity.pk}, sk = {entity.sk}")

------ All Data ------

entity.type = 'student', pk = s-1, sk = s-1

entity.type = 'enrolled_course', pk = s-1, sk = c-1

entity.type = 'student', pk = s-2, sk = s-2

entity.type = 'enrolled_course', pk = s-2, sk = c-1

entity.type = 'enrolled_course', pk = s-2, sk = c-2

entity.type = 'student', pk = s-3, sk = s-3

entity.type = 'enrolled_course', pk = s-3, sk = c-2

entity.type = 'course', pk = c-1, sk = c-1

entity.type = 'course', pk = c-2, sk = c-2

# Find all Students enrolled in the Given Course

In [10]:
rprint("--- Student in Math course ---")
for student in op.find_all_student_in_given_course(course_id="c-1"):
    student = Student.get(hash_key=student.pk, range_key=student.pk)
    rprint(student.name)
    
rprint("--- Bob enrolled course ---")
for course in op.find_all_enrolled_course(student_id="s-2"):
    course = Course.get(hash_key=course.sk, range_key=course.sk)
    rprint(course.name)

--- Student in Math course ---

Alice

Bob

--- Bob enrolled course ---

Math

Science